#  📊 YouTube Trending Videos - Data Cleaning

The purpose of this notebook is to clean and prepare the YouTube dataset for further analysis. 
The dataset has been downloaded from Kaggle: https://www.kaggle.com/datasets/datasnaek/youtube-new?resource=download&select=USvideos.csv
and contains information on the most viral videos of recent years, in this case focusing on videos from the United States.
The steps to follow will be:

- Removal of duplicates.
- Handling of null values.
- Conversion of dates and times.
- Creation of useful derived metrics for analysis.
- Verify that all data types are correct for further analysis.

**Dataset original:** `USvideos.csv`  
**Autor:** Diego Aladrén Mateo

## Import libraries

In [5]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os

# Settings to view all columns
pd.set_option('display.max_columns', None)

# Seaborn aesthetic configuration
sns.set(style='whitegrid')

## Load dataset

In [7]:
# Relative path of the project
csv_path = os.path.join("..", "data", "USvideos.csv")

# Load the dataset
df = pd.read_csv(csv_path)

# Show the first lines
df.head()

,video_id,trending_date,title,channel_title,category_id,publish_time,tags,views,likes,dislikes,comment_count,thumbnail_link,comments_disabled,ratings_disabled,video_error_or_removed,description
0,2kyS6SvSYSE,17.14.11,WE WANT TO TALK ABOUT OUR MARRIAGE,CaseyNeistat,22,2017-11-13T17:13:01.000Z,SHANtell martin,748374,57527,2966,15954,https://i.ytimg.com/vi/2kyS6SvSYSE/default.jpg,False,False,False,SHANTELL'S CHANNEL - https://www.youtube.com/s...
1,1ZAPwfrtAFY,17.14.11,The Trump Presidency: Last Week Tonight with J...,LastWeekTonight,24,2017-11-13T07:30:00.000Z,"last week tonight trump presidency|""last week ...",2418783,97185,6146,12703,https://i.ytimg.com/vi/1ZAPwfrtAFY/default.jpg,False,False,False,"One year after the presidential election, John..."
2,5qpjK5DgCt4,17.14.11,"Racist Superman | Rudy Mancuso, King Bach & Le...",Rudy Mancuso,23,2017-11-12T19:05:24.000Z,"racist superman|""rudy""|""mancuso""|""king""|""bach""...",3191434,146033,5339,8181,https://i.ytimg.com/vi/5qpjK5DgCt4/default.jpg,False,False,False,WATCH MY PREVIOUS VIDEO ▶ \n\nSUBSCRIBE ► http...
3,puqaWrEC7tY,17.14.11,Nickelback Lyrics: Real or Fake?,Good Mythical Morning,24,2017-11-13T11:00:04.000Z,"rhett and link|""gmm""|""good mythical morning""|""...",343168,10172,666,2146,https://i.ytimg.com/vi/puqaWrEC7tY/default.jpg,False,False,False,Today we find out if Link is a Nickelback amat...
4,d380meD0W0M,17.14.11,I Dare You: GOING BALD!?,nigahiga,24,2017-11-12T18:01:41.000Z,"ryan|""higa""|""higatv""|""nigahiga""|""i dare you""|""...",2095731,132235,1989,17518,https://i.ytimg.com/vi/d380meD0W0M/default.jpg,False,False,False,I know it's been a while since we did this sho...


## Initial exploration

In [13]:
# Dataset dimensions
print("Rows and columns: ", df.shape)

# General information
df.info

# Check null values
print("\nNull values per column:\n", df.isnull().sum())

Rows and columns:  (40949, 16)

Null values per column:
 video_id                    0
trending_date               0
title                       0
channel_title               0
category_id                 0
publish_time                0
tags                        0
views                       0
likes                       0
dislikes                    0
comment_count               0
thumbnail_link              0
comments_disabled           0
ratings_disabled            0
video_error_or_removed      0
description               570
dtype: int64


# Basic data cleaning

In this dataset, we only find null values in the description column.
It doesn't seem to have many errors, but let's do some basic cleaning just in case.

In [14]:
# Remove duplicates
df.drop_duplicates()

# Remove critical rows with null values (none should be removed based on what we have seen above).
df = df.dropna(subset=['title', 'channel_title', 'category_id'])

# Check results
print(df.shape)

(40949, 16)


# Column formatting and transformation

In [15]:
# Converting date columns
df['trending_date'] = pd.to_datetime(df['trending_date'], format='%y.%d.%m', errors='coerce')
df['publish_time'] = pd.to_datetime(df['publish_time'], errors='coerce')

# Creating auxiliar columns
df['publish_date'] = df['publish_time'].dt.date
df['publish_hour'] = df['publish_time'].dt.hour

# Show changes
df[['trending_date','publish_time','publish_date','publish_hour']].head()

,trending_date,publish_time,publish_date,publish_hour
0,2017-11-14,2017-11-13 17:13:01+00:00,2017-11-13,17
1,2017-11-14,2017-11-13 07:30:00+00:00,2017-11-13,7
2,2017-11-14,2017-11-12 19:05:24+00:00,2017-11-12,19
3,2017-11-14,2017-11-13 11:00:04+00:00,2017-11-13,11
4,2017-11-14,2017-11-12 18:01:41+00:00,2017-11-12,18


# Derived metrics

In [16]:
# Interaction rate
df['like_ratio'] = df['likes'] / df['views']
df['comment_ratio'] = df['comment_count'] / df['views']

# Check
df[['views','likes','comment_count','like_ratio','comment_ratio']].head()

,views,likes,comment_count,like_ratio,comment_ratio
0,748374,57527,15954,0.076869,0.021318
1,2418783,97185,12703,0.040179,0.005252
2,3191434,146033,8181,0.045758,0.002563
3,343168,10172,2146,0.029641,0.006253
4,2095731,132235,17518,0.063097,0.008359


Before saving the cleaned dataset, we will perform one last check to ensure that the data types are correct.

In [20]:
pd.DataFrame(df.dtypes, columns=['Data type'])


,Data type
video_id,object
trending_date,datetime64[ns]
title,object
channel_title,object
category_id,int64
publish_time,"datetime64[ns, UTC]"
tags,object
views,int64
likes,int64
dislikes,int64


The ‘publish_date’ column should be converted to datetime64 to analyze by publication date. It is currently an object (derived from publish_time).

In [23]:
df['publish_date'] = pd.to_datetime(df['publish_date'], errors='coerce')

pd.DataFrame(df.dtypes, columns=['Data type'])

,Data type
video_id,object
trending_date,datetime64[ns]
title,object
channel_title,object
category_id,int64
publish_time,"datetime64[ns, UTC]"
tags,object
views,int64
likes,int64
dislikes,int64


Now let's put the tags as lists to identify keywords.

In [25]:
df['tags_list'] = df['tags'].str.split('|')

df['tags_list'].head()

0                                    [SHANtell martin]
1    [last week tonight trump presidency, "last wee...
2    [racist superman, "rudy", "mancuso", "king", "...
3    [rhett and link, "gmm", "good mythical morning...
4    [ryan, "higa", "higatv", "nigahiga", "i dare y...
Name: tags_list, dtype: object

# Save clean dataset

Now that we have the dataset cleaned up and ready to use, let's save it.

In [27]:
df.head()

,video_id,trending_date,title,channel_title,category_id,publish_time,tags,views,likes,dislikes,comment_count,thumbnail_link,comments_disabled,ratings_disabled,video_error_or_removed,description,publish_date,publish_hour,like_ratio,comment_ratio,tags_list
0,2kyS6SvSYSE,2017-11-14,WE WANT TO TALK ABOUT OUR MARRIAGE,CaseyNeistat,22,2017-11-13 17:13:01+00:00,SHANtell martin,748374,57527,2966,15954,https://i.ytimg.com/vi/2kyS6SvSYSE/default.jpg,False,False,False,SHANTELL'S CHANNEL - https://www.youtube.com/s...,2017-11-13,17,0.076869,0.021318,[SHANtell martin]
1,1ZAPwfrtAFY,2017-11-14,The Trump Presidency: Last Week Tonight with J...,LastWeekTonight,24,2017-11-13 07:30:00+00:00,"last week tonight trump presidency|""last week ...",2418783,97185,6146,12703,https://i.ytimg.com/vi/1ZAPwfrtAFY/default.jpg,False,False,False,"One year after the presidential election, John...",2017-11-13,7,0.040179,0.005252,"[last week tonight trump presidency, ""last wee..."
2,5qpjK5DgCt4,2017-11-14,"Racist Superman | Rudy Mancuso, King Bach & Le...",Rudy Mancuso,23,2017-11-12 19:05:24+00:00,"racist superman|""rudy""|""mancuso""|""king""|""bach""...",3191434,146033,5339,8181,https://i.ytimg.com/vi/5qpjK5DgCt4/default.jpg,False,False,False,WATCH MY PREVIOUS VIDEO ▶ \n\nSUBSCRIBE ► http...,2017-11-12,19,0.045758,0.002563,"[racist superman, ""rudy"", ""mancuso"", ""king"", ""..."
3,puqaWrEC7tY,2017-11-14,Nickelback Lyrics: Real or Fake?,Good Mythical Morning,24,2017-11-13 11:00:04+00:00,"rhett and link|""gmm""|""good mythical morning""|""...",343168,10172,666,2146,https://i.ytimg.com/vi/puqaWrEC7tY/default.jpg,False,False,False,Today we find out if Link is a Nickelback amat...,2017-11-13,11,0.029641,0.006253,"[rhett and link, ""gmm"", ""good mythical morning..."
4,d380meD0W0M,2017-11-14,I Dare You: GOING BALD!?,nigahiga,24,2017-11-12 18:01:41+00:00,"ryan|""higa""|""higatv""|""nigahiga""|""i dare you""|""...",2095731,132235,1989,17518,https://i.ytimg.com/vi/d380meD0W0M/default.jpg,False,False,False,I know it's been a while since we did this sho...,2017-11-12,18,0.063097,0.008359,"[ryan, ""higa"", ""higatv"", ""nigahiga"", ""i dare y..."


In [28]:

clean_csv_path = os.path.join("..", "data", "USvideos_clean.csv")

# save CSV
df.to_csv(clean_csv_path, index=False)

print("Clean dataset saved in:", clean_csv_path)


Clean dataset saved in: ..\data\USvideos_clean.csv
